# dependencies

In [1]:
# general
import requests 
import os
import re 

# data
import pandas as pd

# local dependencies
import dirtools
from nstools import NationStatesAPI
from issue_results_api import IssuePredictionAPI

In [3]:
def pipe(nationName = 'north_kalandia', 
         headers = {'User-Agent':'Nation Info Getter', "X-Password": "koraxhun123"},
         urlStats="https://www.nationstates.net/cgi-bin/api.cgi?nation=north_kalandia&q=census;scale=all;mode=score"
):
    # get issues

    baseDir = os.getcwd()
    dirs = dirtools.Dirs(baseDir, nationName)
    nationStatesClass = NationStatesAPI(dirs, headers)
    urlIssues = nationStatesClass.generate_issues_url()
    issuesList = nationStatesClass.get_nation_issues(dirs.issuesDir, urlIssues)

    ipa = IssuePredictionAPI(dirs)
    issuePredictions = {}
    for issue in issuesList:
        ipa.get_predictions(issue)
        issuePredictions[issue] = ipa.process_predictions()
    return issuePredictions

In [4]:
pipe()

creating directory:  /home/alex/Documents/ns/north_kalandia/2020-12-29
creating directory:  /home/alex/Documents/ns/north_kalandia/2020-12-29/stats
creating directory:  /home/alex/Documents/ns/north_kalandia/2020-12-29/issues


{'1303': [{'result summary': '1. athletic teenagers are among the wealthiest members of society</td>',
   'Economy': [-0.45, 1.45, 0.2613],
   'Wealth Gaps': [0.0, 188.25, 0.3414],
   'Death Rate': [-1.69, 2.83, 0.0069],
   'Eco-Friendliness': [-30.69, 151.88, 0.8086],
   'Industry: Automobile Manufacturing': [-73.13, 101.4, -0.3801],
   'Industry: Cheese Exports': [-37.95, 137.8, 0.3682],
   'Industry: Basket Weaving': [-34.59, 163.41, 1.1531],
   'Industry: Information Technology': [-143.78, 535.44, -3.3133],
   'Industry: Pizza Delivery': [-92.92, 145.37, 0.4707],
   'Industry: Trout Fishing': [-81.98, 383.52, 3.178],
   'Industry: Arms Manufacturing': [-137.97, 217.83, -3.6081],
   'Sector: Agriculture': [-34.89, 157.13, 0.0401],
   'Industry: Beverage Sales': [-168.01, 140.74, -0.9476],
   'Industry: Timber Woodchipping': [-68.84, 102.5, -0.1896],
   'Industry: Mining': [-107.8, 133.34, -0.811],
   'Industry: Insurance': [-52.06, 121.0, 0.4913],
   'Industry: Furniture Restoration

In [10]:
issuesList

['1303', '1081', '1101', '619', '1312']

In [11]:
for issue in issuePredictions:
    current = issuePredictions[issue]
    print(issue, ': ', end='')
    for i in range(len(current)):
        
        choice = current[i]
        del choice['result summary']
        exec(f"df_{issue}_{i} = pd.DataFrame(choice)")
    print(i)

1303 : 3
1081 : 2
1101 : 4
619 : 4
1312 : 3


In [12]:
df_1303_1

,Economy,Wealth Gaps,Death Rate,Eco-Friendliness,Social Conservatism,Nudity,Industry: Automobile Manufacturing,Industry: Cheese Exports,Industry: Basket Weaving,Industry: Information Technology,...,Human Development Index,Primitiveness,Scientific Advancement,Average Income,Average Income of Poor,Average Income of Rich,Public Education,Crime,Foreign Aid,Average Disposable Income
0,-1.3400,-241.9400,-2.010,-86.9400,-0.6900,0.0000,-119.1600,-136.8600,-155.7200,-136.0900,...,-0.2200,-24.5000,0.0700,-1199.6900,-702.8300,-12254.0000,-118.2800,-0.2400,-50.1500,-992.8200
1,0.5500,-0.0000,2.270,4.5900,-0.0000,0.2500,40.8300,36.3900,69.7900,216.1900,...,1.2600,-0.0700,24.1400,531.5400,670.8400,1076.0000,12.9600,-0.0000,2.2200,500.7100
2,-0.1654,-0.4172,-0.012,-1.5456,-0.0003,0.0001,-3.4366,-3.7675,-4.3515,71.0517,...,0.0656,-1.7576,1.5433,-51.6249,121.9986,-1009.8991,-4.2263,-0.0112,-0.6846,-29.3538


# get nation stats

In [28]:
urlStats = "https://www.nationstates.net/cgi-bin/api.cgi?nation=north_kalandia&q=census;scale=all;mode=score"
headers = {'User-Agent':'Nation Info Getter', "X-Password": "koraxhun123"}


In [3]:
stats = nationStatesClass.get_nation_stats(url=urlStats, headers=headers)


<NATION id="north_kalandia">
<CENSUS>
<SCALE id="0">
<SCORE>67.33</SCORE>
</SCALE>
<SCALE id="1">
<SCORE>90.78</SCORE>
</SCALE>
<SCALE id="2">
<SCORE>38.10</SCORE>
</SCALE>
<SCALE id="3">
<SCORE>9339000000</SCORE>
</SCALE>
<SCALE id="4">
<SCORE>3.00</SCORE>
</SCALE>
<SCALE id="5">
<SCORE>20.86</SCORE>
</SCALE>
<SCALE id="6">
<SCORE>12.49</SCORE>
</SCALE>
<SCALE id="7">
<SCORE>-5.48</SCORE>
</SCALE>
<SCALE id="8">
<SCORE>40.00</SCORE>
</SCALE>
<SCALE id="9">
<SCORE>197.00</SCORE>
</SCALE>
<SCALE id="10">
<SCORE>-20.30</SCORE>
</SCALE>
<SCALE id="11">
<SCORE>2688.97</SCORE>
</SCALE>
<SCALE id="12">
<SCORE>127.18</SCORE>
</SCALE>
<SCALE id="13">
<SCORE>27298.48</SCORE>
</SCALE>
<SCALE id="14">
<SCORE>-13.62</SCORE>
</SCALE>
<SCALE id="15">
<SCORE>-7.16</SCORE>
</SCALE>
<SCALE id="16">
<SCORE>26844.27</SCORE>
</SCALE>
<SCALE id="17">
<SCORE>3031.90</SCORE>
</SCALE>
<SCALE id="18">
<SCORE>-12.73</SCORE>
</SCALE>
<SCALE id="19">
<SCORE>181.69</SCORE>
</SCALE>
<SCALE id="20">
<SCORE>1544.34</

In [30]:
stats

{}

In [38]:
with open('censusids.txt') as f:
     censusids = [line.rstrip() for line in f]

In [39]:
for c in df_1303_1.columns:
    if c not in censusids:
        print(c)

In [40]:
censusids    

['Civil Rights',
 'Economy',
 'Political Freedom',
 'Population',
 'Wealth Gaps',
 'Death Rate',
 'Compassion',
 'Eco-Friendliness',
 'Social Conservatism',
 'Nudity',
 'Industry: Automobile Manufacturing',
 'Industry: Cheese Exports',
 'Industry: Basket Weaving',
 'Industry: Information Technology',
 'Industry: Pizza Delivery',
 'Industry: Trout Fishing',
 'Industry: Arms Manufacturing',
 'Sector: Agriculture',
 'Industry: Beverage Sales',
 'Industry: Timber Woodchipping',
 'Industry: Mining',
 'Industry: Insurance',
 'Industry: Furniture Restoration',
 'Industry: Retail',
 'Industry: Book Publishing',
 'Industry: Gambling',
 'Sector: Manufacturing',
 'Government Size',
 'Welfare',
 'Public Healthcare',
 'Law Enforcement',
 'Business Subsidization',
 'Religiousness',
 'Income Equality',
 'Niceness',
 'Rudeness',
 'Intelligence',
 'Ignorance',
 'Political Apathy',
 'Health',
 'Cheerfulness',
 'Weather',
 'Compliance',
 'Safety',
 'Lifespan',
 'Ideological Radicality',
 'Defense Forces'

In [18]:
fin = """<NATION id="north_kalandia">
<CENSUS>
<SCALE id="0">
<SCORE>67.33</SCORE>
</SCALE>
<SCALE id="1">
<SCORE>90.78</SCORE>
</SCALE>
<SCALE id="2">
<SCORE>38.10</SCORE>
</SCALE>
<SCALE id="3">
<SCORE>9339000000</SCORE>
</SCALE>
<SCALE id="4">
<SCORE>3.00</SCORE>
</SCALE>
<SCALE id="5">
<SCORE>20.86</SCORE>
</SCALE>
<SCALE id="6">
<SCORE>12.49</SCORE>
</SCALE>
<SCALE id="7">
<SCORE>-5.48</SCORE>
</SCALE>
<SCALE id="8">
<SCORE>40.00</SCORE>
</SCALE>
<SCALE id="9">
<SCORE>197.00</SCORE>
</SCALE>
<SCALE id="10">
<SCORE>-20.30</SCORE>
</SCALE>
<SCALE id="11">
<SCORE>2688.97</SCORE>
</SCALE>
<SCALE id="12">
<SCORE>127.18</SCORE>
</SCALE>
<SCALE id="13">
<SCORE>27298.48</SCORE>
</SCALE>
<SCALE id="14">
<SCORE>-13.62</SCORE>
</SCALE>
<SCALE id="15">
<SCORE>-7.16</SCORE>
</SCALE>
<SCALE id="16">
<SCORE>26844.27</SCORE>
</SCALE>
<SCALE id="17">
<SCORE>3031.90</SCORE>
</SCALE>
<SCALE id="18">
<SCORE>-12.73</SCORE>
</SCALE>
<SCALE id="19">
<SCORE>181.69</SCORE>
</SCALE>
<SCALE id="20">
<SCORE>1544.34</SCORE>
</SCALE>
<SCALE id="21">
<SCORE>-17.29</SCORE>
</SCALE>
<SCALE id="22">
<SCORE>-11.05</SCORE>
</SCALE>
<SCALE id="23">
<SCORE>11241.89</SCORE>
</SCALE>
<SCALE id="24">
<SCORE>6913.19</SCORE>
</SCALE>
<SCALE id="25">
<SCORE>-13.83</SCORE>
</SCALE>
<SCALE id="26">
<SCORE>26971.45</SCORE>
</SCALE>
<SCALE id="27">
<SCORE>28.30</SCORE>
</SCALE>
<SCALE id="28">
<SCORE>-25.50</SCORE>
</SCALE>
<SCALE id="29">
<SCORE>4598.95</SCORE>
</SCALE>
<SCALE id="30">
<SCORE>9481.79</SCORE>
</SCALE>
<SCALE id="31">
<SCORE>2043.98</SCORE>
</SCALE>
<SCALE id="32">
<SCORE>4.67</SCORE>
</SCALE>
<SCALE id="33">
<SCORE>33.30</SCORE>
</SCALE>
<SCALE id="34">
<SCORE>0.35</SCORE>
</SCALE>
<SCALE id="35">
<SCORE>128.99</SCORE>
</SCALE>
<SCALE id="36">
<SCORE>66.11</SCORE>
</SCALE>
<SCALE id="37">
<SCORE>1.07</SCORE>
</SCALE>
<SCALE id="38">
<SCORE>46.97</SCORE>
</SCALE>
<SCALE id="39">
<SCORE>16.75</SCORE>
</SCALE>
<SCALE id="40">
<SCORE>36.67</SCORE>
</SCALE>
<SCALE id="41">
<SCORE>-23.00</SCORE>
</SCALE>
<SCALE id="42">
<SCORE>89.68</SCORE>
</SCALE>
<SCALE id="43">
<SCORE>125.16</SCORE>
</SCALE>
<SCALE id="44">
<SCORE>91.14</SCORE>
</SCALE>
<SCALE id="45">
<SCORE>14.85</SCORE>
</SCALE>
<SCALE id="46">
<SCORE>14478.19</SCORE>
</SCALE>
<SCALE id="47">
<SCORE>4.84</SCORE>
</SCALE>
<SCALE id="48">
<SCORE>68.33</SCORE>
</SCALE>
<SCALE id="49">
<SCORE>72.10</SCORE>
</SCALE>
<SCALE id="50">
<SCORE>-80.53</SCORE>
</SCALE>
<SCALE id="51">
<SCORE>45.09</SCORE>
</SCALE>
<SCALE id="52">
<SCORE>90.21</SCORE>
</SCALE>
<SCALE id="53">
<SCORE>502.44</SCORE>
</SCALE>
<SCALE id="54">
<SCORE>-6.00</SCORE>
</SCALE>
<SCALE id="55">
<SCORE>45.00</SCORE>
</SCALE>
<SCALE id="56">
<SCORE>71.89</SCORE>
</SCALE>
<SCALE id="57">
<SCORE>3804.07</SCORE>
</SCALE>
<SCALE id="58">
<SCORE>12.62</SCORE>
</SCALE>
<SCALE id="59">
<SCORE>0.16</SCORE>
</SCALE>
<SCALE id="60">
<SCORE>87.00</SCORE>
</SCALE>
<SCALE id="61">
<SCORE>6.74</SCORE>
</SCALE>
<SCALE id="62">
<SCORE>67.08</SCORE>
</SCALE>
<SCALE id="63">
<SCORE>3.75</SCORE>
</SCALE>
<SCALE id="64">
<SCORE>450.63</SCORE>
</SCALE>
<SCALE id="65">
<SCORE>10.00</SCORE>
</SCALE>
<SCALE id="66">
<SCORE>0.00</SCORE>
</SCALE>
<SCALE id="67">
<SCORE>40.15</SCORE>
</SCALE>
<SCALE id="68">
<SCORE>75.65</SCORE>
</SCALE>
<SCALE id="69">
<SCORE>-377.89</SCORE>
</SCALE>
<SCALE id="70">
<SCORE>378.67</SCORE>
</SCALE>
<SCALE id="71">
<SCORE>135.70</SCORE>
</SCALE>
<SCALE id="72">
<SCORE>125706.56</SCORE>
</SCALE>
<SCALE id="73">
<SCORE>69836.98</SCORE>
</SCALE>
<SCALE id="74">
<SCORE>209720</SCORE>
</SCALE>
<SCALE id="75">
<SCORE>5109.95</SCORE>
</SCALE>
<SCALE id="76">
<SCORE>1.173974e+15</SCORE>
</SCALE>
<SCALE id="77">
<SCORE>1.79</SCORE>
</SCALE>
<SCALE id="78">
<SCORE>-16.73</SCORE>
</SCALE>
<SCALE id="79">
<SCORE>471900000000</SCORE>
</SCALE>
<SCALE id="80">
<SCORE>5.07477452992</SCORE>
</SCALE>
<SCALE id="81">
<SCORE>9306000000</SCORE>
</SCALE>
<SCALE id="82">
<SCORE>0</SCORE>
</SCALE>
<SCALE id="83">
<SCORE>0</SCORE>
</SCALE>
<SCALE id="84">
<SCORE>0</SCORE>
</SCALE>
<SCALE id="85">
<SCORE>35072.13</SCORE>
</SCALE>
<SCALE id="86">
<SCORE>3.28</SCORE>
</SCALE>
</CENSUS>
</NATION>"""

In [34]:
lines = fin.split('\n')
prev = False
stats = {}
for line in lines:
    pattern = re.findall('id="\d+"', line)
    if pattern != []:
        idrow = True
        cur = pattern[0]
        censusid = re.findall('\d+', cur)[0]
    else:
        idrow = False
        if prev == True:
            score = re.findall('>.*<', line)[0]
            score = score[1:-1]
            score = float(score)
            stats[censusid] = score
    prev = idrow

In [35]:
stats

{'0': 67.33,
 '1': 90.78,
 '2': 38.1,
 '3': 9339000000.0,
 '4': 3.0,
 '5': 20.86,
 '6': 12.49,
 '7': -5.48,
 '8': 40.0,
 '9': 197.0,
 '10': -20.3,
 '11': 2688.97,
 '12': 127.18,
 '13': 27298.48,
 '14': -13.62,
 '15': -7.16,
 '16': 26844.27,
 '17': 3031.9,
 '18': -12.73,
 '19': 181.69,
 '20': 1544.34,
 '21': -17.29,
 '22': -11.05,
 '23': 11241.89,
 '24': 6913.19,
 '25': -13.83,
 '26': 26971.45,
 '27': 28.3,
 '28': -25.5,
 '29': 4598.95,
 '30': 9481.79,
 '31': 2043.98,
 '32': 4.67,
 '33': 33.3,
 '34': 0.35,
 '35': 128.99,
 '36': 66.11,
 '37': 1.07,
 '38': 46.97,
 '39': 16.75,
 '40': 36.67,
 '41': -23.0,
 '42': 89.68,
 '43': 125.16,
 '44': 91.14,
 '45': 14.85,
 '46': 14478.19,
 '47': 4.84,
 '48': 68.33,
 '49': 72.1,
 '50': -80.53,
 '51': 45.09,
 '52': 90.21,
 '53': 502.44,
 '54': -6.0,
 '55': 45.0,
 '56': 71.89,
 '57': 3804.07,
 '58': 12.62,
 '59': 0.16,
 '60': 87.0,
 '61': 6.74,
 '62': 67.08,
 '63': 3.75,
 '64': 450.63,
 '65': 10.0,
 '66': 0.0,
 '67': 40.15,
 '68': 75.65,
 '69': -377.89,

In [41]:
metricStats = {}
for k in stats:
    metricStats[censusids[int(k)]]= stats[k]

In [42]:
metricStats

{'Civil Rights': 67.33,
 'Economy': 90.78,
 'Political Freedom': 38.1,
 'Population': 9339000000.0,
 'Wealth Gaps': 3.0,
 'Death Rate': 20.86,
 'Compassion': 12.49,
 'Eco-Friendliness': -5.48,
 'Social Conservatism': 40.0,
 'Nudity': 197.0,
 'Industry: Automobile Manufacturing': -20.3,
 'Industry: Cheese Exports': 2688.97,
 'Industry: Basket Weaving': 127.18,
 'Industry: Information Technology': 27298.48,
 'Industry: Pizza Delivery': -13.62,
 'Industry: Trout Fishing': -7.16,
 'Industry: Arms Manufacturing': 26844.27,
 'Sector: Agriculture': 3031.9,
 'Industry: Beverage Sales': -12.73,
 'Industry: Timber Woodchipping': 181.69,
 'Industry: Mining': 1544.34,
 'Industry: Insurance': -17.29,
 'Industry: Furniture Restoration': -11.05,
 'Industry: Retail': 11241.89,
 'Industry: Book Publishing': 6913.19,
 'Industry: Gambling': -13.83,
 'Sector: Manufacturing': 26971.45,
 'Government Size': 28.3,
 'Welfare': -25.5,
 'Public Healthcare': 4598.95,
 'Law Enforcement': 9481.79,
 'Business Subsid

# analysis

In [53]:
proc_df_1303_1 = df_1303_1.copy()
for c in proc_df_1303_1.columns:
    metricStats[c]
    proc_df_1303_1[c]
    proc_df_1303_1[c] = proc_df_1303_1[c]/metricStats[c]

In [54]:
proc_df_1303_1

,Economy,Wealth Gaps,Death Rate,Eco-Friendliness,Social Conservatism,Nudity,Industry: Automobile Manufacturing,Industry: Cheese Exports,Industry: Basket Weaving,Industry: Information Technology,...,Human Development Index,Primitiveness,Scientific Advancement,Average Income,Average Income of Poor,Average Income of Rich,Public Education,Crime,Foreign Aid,Average Disposable Income
0,-0.014761,-80.646667,-0.096357,15.864964,-0.017250,0.000000e+00,5.869951,-0.050897,-1.224406,-0.004985,...,-0.002908,0.064834,0.000185,-0.009544,-0.010064,-0.058430,-0.023147,-0.134078,2.997609,-0.028308
1,0.006059,-0.000000,0.108821,-0.837591,-0.000000,1.269036e-03,-2.011330,0.013533,0.548750,0.007919,...,0.016656,0.000185,0.063749,0.004228,0.009606,0.005131,0.002536,-0.000000,-0.132696,0.014277
2,-0.001822,-0.139067,-0.000575,0.282044,-0.000007,5.076142e-07,0.169291,-0.001401,-0.034215,0.002603,...,0.000867,0.004651,0.004076,-0.000411,0.001747,-0.004815,-0.000827,-0.006257,0.040921,-0.000837


In [57]:
proc_df_1303_1['Wealth Gaps']

0   -80.646667
1    -0.000000
2    -0.139067
Name: Wealth Gaps, dtype: float64

In [62]:
final = proc_df_1303_1.loc[1,:]

In [65]:
final.sort_values()*100

Industry: Trout Fishing              -1261.871508
Industry: Pizza Delivery              -485.976505
Industry: Beverage Sales              -460.565593
Industry: Furniture Restoration       -371.312217
Industry: Insurance                   -349.161365
Industry: Automobile Manufacturing    -201.133005
Industry: Gambling                    -157.194505
Eco-Friendliness                       -83.759124
Welfare                                -44.470588
Foreign Aid                            -13.269576
Freedom From Taxation                   -0.173848
Ignorance                               -0.000000
Weaponization                           -0.000000
Obesity                                 -0.000000
Economic Freedom                        -0.000000
Wealth Gaps                             -0.000000
Crime                                   -0.000000
Social Conservatism                     -0.000000
Rudeness                                 0.015505
Primitiveness                            0.018524
